# LSTM Models with 3 different languages

### Create the configuration for the experiment

All the languages with use the same model configuration for comparing performances across them

In [1]:
%pip install conllu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras

keras.utils.set_random_seed(50)

/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next r

In [3]:
from data.preprocessor import DataPreprocessor, DataPreprocessorConfig
from trainer.trainer import TrainerConfig, Trainer
from models.base_model import ModelConfig

preprocessor_config = DataPreprocessorConfig(
    padding_type="post",
    truncation_type="post",
    remove_long_sentences=True,
    max_sequence_length=100,
)

training_config = TrainerConfig(
    epochs=20,
    batch_size=64,
    early_stopping_patience=3,
    learning_rate=1e-3,
    model_dir="saved_models",
    save_best_only=True,
)

model_config = ModelConfig(
    embedding_dim=128,
    lstm_units=128,
    bidirectional=True,
    dropout_rate=0.3,
    training_config=training_config,
)

## English

### Preprocesses the data

In [4]:
from utils import load_data

train_data, dev_data, test_data = load_data("english")

In [5]:
preprocessor = DataPreprocessor(preprocessor_config)

In [6]:
X_train, y_train = preprocessor.process_data_to_pad_sequences(
    train_data, is_train_dataset=True
)
X_dev, y_dev = preprocessor.process_data_to_pad_sequences(
    dev_data, is_train_dataset=False
)
X_test, y_test = preprocessor.process_data_to_pad_sequences(
    test_data, is_train_dataset=False
)

Sample sentence: Al - Zaman : American forces killed Shaikh Abdullah al - Ani , the preacher at the mosque in the town of Qaim , near the Syrian border .
Sample tags: PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  264    16  6152    45   293   709  1150  4860 18886   592    16 18792
     4     3  7335    35     3  7509    10     3   486     8  8930     4
   751     3  2357  1642     2     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
Sample

### Initilize the model with the config

In [7]:
from models.lstm_model import LSTMModel

english_lstm = LSTMModel(
    model_config,
    preprocessor.vocab_size,
    preprocessor.num_tags,
    preprocessor_config.max_sequence_length,
)
english_lstm.build_model()
english_lstm.compile_model()

print("Model summary:\n")
print(english_lstm.get_model().summary())

Model summary:



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     2,518,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,786,579 (10.63 MB)

 Trainable params: 2,786,579 (10.63 MB)

 Non-trainable params: 0 (0.00 B)

None


### Training the model

In [8]:
# Initialize trainer
trainer = Trainer(training_config, english_lstm, preprocessor)

In [9]:
# Train the model
print("Training model...\n")
trainer.train((X_train, y_train), (X_dev, y_dev))
print("Training completed.\n")

Training model...

Epoch 1/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - _masked_accuracy: 0.4283 - loss: 1.9550
Epoch 1: val_loss improved from None to 0.47641, saving model to saved_models/BiLSTM_emb128_lstm128_drop30_ep20_bs64_pat3_lr10.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 46s 222ms/step - _masked_accuracy: 0.6379 - loss: 1.2329 - val__masked_accuracy: 0.8616 - val_loss: 0.4764
Epoch 2/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - _masked_accuracy: 0.9137 - loss: 0.3148
Epoch 2: val_loss improved from 0.47641 to 0.31451, saving model to saved_models/BiLSTM_emb128_lstm128_drop30_ep20_bs64_pat3_lr10.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 47s 242ms/step - _masked_accuracy: 0.9311 - loss: 0.2520 - val__masked_accuracy: 0.9062 - val_loss: 0.3145
Epoch 3/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - _masked_accuracy: 0.9591 - loss: 0.1478
Epoch 3: val_loss did not improve from 0.31451
196/196 ━━━━━━━━━━━━━━━━━━━━ 49s 252ms/step - _masked_accuracy: 0.9640 - loss: 0.1306 - val__masked_accur

### Testing the model

In [10]:
from evaluator.evaluator import Evaluator

evaluator = Evaluator(english_lstm, preprocessor)

In [11]:
test_metrics = evaluator.evaluate(X_test, y_test, "Test")


Test Set Evaluation:
Accuracy: 0.9102

Detailed Classification Report:
              precision    recall  f1-score   support

         ADJ       0.85      0.87      0.86      1788
         ADP       0.92      0.97      0.94      2029
         ADV       0.90      0.85      0.88      1191
         AUX       0.97      0.98      0.97      1543
       CCONJ       0.99      0.99      0.99       736
         DET       0.98      0.98      0.98      1897
        INTJ       1.00      0.59      0.74       121
        NOUN       0.84      0.90      0.87      4123
         NUM       0.70      0.79      0.74       542
        PART       0.94      0.96      0.95       649
        PRON       0.98      0.98      0.98      2165
       PROPN       0.87      0.67      0.76      2075
       PUNCT       0.99      0.99      0.99      3096
       SCONJ       0.88      0.73      0.80       384
         SYM       0.80      0.72      0.76       109
        VERB       0.88      0.93      0.91      2606
         

### Inference on new sentences

In [12]:
from inference.predictor import Predictor

# Create predictor for inference
predictor = Predictor(english_lstm.get_model(), preprocessor)

In [13]:
example_sentence = "Today it is cloudy"
predicted_tags = predictor.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")


Example prediction:
Sentence: Today it is cloudy
Predicted tags: NOUN PRON AUX ADJ


## Spanish

In [14]:
from utils import load_data

train_data_es, dev_data_es, test_data_es = load_data("spanish")

In [15]:
preprocessor_es = DataPreprocessor(preprocessor_config)

In [16]:
X_train_es, y_train_es = preprocessor_es.process_data_to_pad_sequences(
    train_data_es, is_train_dataset=True
)
X_dev_es, y_dev_es = preprocessor_es.process_data_to_pad_sequences(
    dev_data_es, is_train_dataset=False
)
X_test_es, y_test_es = preprocessor_es.process_data_to_pad_sequences(
    test_data_es, is_train_dataset=False
)

Sample sentence: Además se le pediría a las empresas interesadas en prestar el servicio que se hagan cargo de la señalización y la cartelería que contiene información para los usuarios .
Sample tags: ADV PRON PRON VERB ADP DET NOUN ADJ ADP VERB DET NOUN SCONJ PRON VERB NOUN ADP DET NOUN CCONJ DET NOUN SCONJ VERB NOUN ADP DET NOUN PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  168    12    37 22916     9    17   656 25241     7  9426     4   247
    10    12  5355   625     2     6  9162     8     6 30478    10  1304
   614    23    11  1380     5     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0   

In [17]:
from models.lstm_model import LSTMModel

es_lstm = LSTMModel(
    model_config,
    preprocessor_es.vocab_size,
    preprocessor_es.num_tags,
    preprocessor_config.max_sequence_length,
)
es_lstm.build_model()
es_lstm.compile_model()

print("Model summary:\n")
print(es_lstm.get_model().summary())

Model summary:



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 128)       │     5,905,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 100, 256)       │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,173,203 (23.55 MB)

 Trainable params: 6,173,203 (23.55 MB)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
trainer = Trainer(training_config, es_lstm, preprocessor_es)

In [ ]:
# Train the model
print("Training model...\n")
trainer.train((X_train_es, y_train_es), (X_dev_es, y_dev_es))
print("Training completed.\n")

Training model...

Epoch 1/20
200/222 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - _masked_accuracy: 0.5030 - loss: 1.6982

In [ ]:
from evaluator.evaluator import Evaluator

evaluator_es = Evaluator(es_lstm, preprocessor_es)

In [ ]:
test_metrics_es = evaluator_es.evaluate(X_test_es, y_test_es, "Test")

### Inference on new sentences

In [ ]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_es = Predictor(es_lstm.get_model(), preprocessor_es)

In [ ]:
example_sentence = "Hoy el tiempo es soleado ."
predicted_tags = predictor_es.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")

## German

In [ ]:
from utils import load_data

train_data_ge, dev_data_ge, test_data_ge = load_data("german")

In [ ]:
preprocessor_ge = DataPreprocessor(preprocessor_config)

In [ ]:
X_train_ge, y_train_ge = preprocessor_ge.process_data_to_pad_sequences(
    train_data_ge, is_train_dataset=True
)
X_dev_ge, y_dev_ge = preprocessor_ge.process_data_to_pad_sequences(
    dev_data_ge, is_train_dataset=False
)
X_test_ge, y_test_ge = preprocessor_ge.process_data_to_pad_sequences(
    test_data_ge, is_train_dataset=False
)

In [ ]:
from models.lstm_model import LSTMModel

ge_lstm = LSTMModel(
    model_config,
    preprocessor_ge.vocab_size,
    preprocessor_ge.num_tags,
    preprocessor_config.max_sequence_length,
)
ge_lstm.build_model()
ge_lstm.compile_model()

print("Model summary:\n")
print(ge_lstm.get_model().summary())

In [ ]:
trainer_ge = Trainer(training_config, ge_lstm, preprocessor_ge)

In [ ]:
# Train the model
print("Training model...\n")
trainer_ge.train((X_train_ge, y_train_ge), (X_dev_ge, y_dev_ge))
print("Training completed.\n")

In [ ]:
from evaluator.evaluator import Evaluator

evaluator_ge = Evaluator(ge_lstm, preprocessor_ge)

In [ ]:
test_metrics_ge = evaluator_ge.evaluate(X_test_ge, y_test_ge, "Test")

### Inference on new sentences

In [ ]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_ge = Predictor(ge_lstm.get_model(), preprocessor_ge)

In [ ]:
example_sentence = "Heute ist es wolkig ."
predicted_tags = predictor_es.predict_sentence(example_sentence)
print(f"\nExample prediction:")
print(f"Sentence: {example_sentence}")
print(f"Predicted tags: {' '.join(predicted_tags)}")